# Trabajo final

En cada celda de código se explica mediante comentarios qué hacer y cómo. 

Respetad los huecos entre comentarios para rellenar con código lo que se pide justo debajo de cada comentario. 

1. Consulta y filtrado de valores nulos (1 puntos)
2. Consultas con Query (2.5 puntos)
3. Consultas indexación booleana (1 punto)
3. Discretización y Variables dummies (1.5 puntos)
4. Union de datasets, agrupamiento y más consultas con grupos (3 puntos)
6. Consultas libres (1 puntos)

## 1 Carga modificación y filtrado de nulos

In [1]:
import pandas as pd

# importar medallas_jjoo.csv
data_medallas = pd.read_csv('medallas_jjoo.csv')

# muestra el tamaño del dataset
print(data_medallas.size)

# mostrar las primeras filas de medallas para comprender su estructura y comprobar la importación
data_medallas.head()

10752


,Year,Host_country,Host_city,Country_Name,Country_Code,Gold,Silver,Bronze
0,1896,Greece,Athens,Great Britain,GBR,2,3,2
1,1896,Greece,Athens,Hungary,HUN,2,1,3
2,1896,Greece,Athens,France,FRA,5,4,2
3,1896,Greece,Athens,United States,USA,11,7,2
4,1896,Greece,Athens,Germany,GER,6,5,2


In [2]:
# comprobar los si existen valores nulos, cuántos son y en qué columnas
print( data_medallas.isnull().sum())

Year             0
Host_country     0
Host_city        0
Country_Name     0
Country_Code    86
Gold             0
Silver           0
Bronze           0
dtype: int64


In [3]:
# Vamos a eliminar la columna country_code que además es la que tiene nulos
data_medallas.drop(columns='Country_Code', inplace=True)

# Las columnas son renombradas a españo: año, host_pais, host_ciudad, pais, oro, plata bronce
# host se refiere a la ciudad o pais organizador de los juegos
data_medallas.columns=['Anio','Host Pais','Host Ciudad', 'Pais','Oro','Plata','Bronce']

# mostramos las primeras filas para comprobar que la eliminación y renombrado de columnas ha sido correcto
data_medallas.head(5)

,Anio,Host Pais,Host Ciudad,Pais,Oro,Plata,Bronce
0,1896,Greece,Athens,Great Britain,2,3,2
1,1896,Greece,Athens,Hungary,2,1,3
2,1896,Greece,Athens,France,5,4,2
3,1896,Greece,Athens,United States,11,7,2
4,1896,Greece,Athens,Germany,6,5,2


In [4]:
# Comprueba si existen filas duplicadas
print(data_medallas.duplicated().sum())

0


## 2. Consultas query 
Realiza las consultas de cada comentario utilizando query. 

Algunas necesitarán alguna acción más, como filtrar una columna y alguna operación, ordenar valores, etc. 

 

In [5]:
# Consulta 1: Mostrar el número y tipo de medallas España en los JJOO de Barcelona 1992
data_medallas.query(
    'Anio == 1992 and `Host Ciudad` == "Barcelona" and Pais == "Spain"'
    )[['Pais','Host Ciudad','Oro','Plata','Bronce']]

,Pais,Host Ciudad,Oro,Plata,Bronce
711,Spain,Barcelona,13,7,2


In [6]:
# Consulta 2: Mostrar cuántas medallas de oro ha ganado españa en toda la historia
print(f"España tiene {data_medallas.query('Pais == "Spain"')['Oro'].sum()} medallas de oro olimpicas")

España tiene 48 medallas de oro olimpicas


In [7]:
# Consulta 3: Mostrar el medallero de Atenas 2004 de forma descendente en columna medallas de oro.
data_medallas.query(
    'Anio == 2004 and `Host Ciudad` == "Athens"'
    )[['Pais','Oro']].sort_values('Oro',ascending=False).set_index('Pais')


,Oro
Pais,
United States,36
China,32
Russia,28
Australia,17
Japan,16
...,...
Mexico,0
Colombia,0
India,0


In [8]:
# Consulta 4: ¿Cuántas veces se han celebrado los JJOO en Francia? Mostrar solamente el número con un mensaje.
print(f"Los JJOO se han celebrado en Francia {data_medallas.query('`Host Pais` == "France"').shape[0]} veces.")

Los JJOO se han celebrado en Francia 48 veces.


In [9]:
# Consulta 5: Mostrar las filas de los paises 'Spain', 'France', 'Germany', 'Italy',  que han ganado más de 10 medallas de oro
data_medallas.query(
    'Pais in ["Spain", "France", "Germany", "Italy"] and Oro > 10'
)[['Anio', 'Host Ciudad', 'Pais', 'Oro']]

,Anio,Host Ciudad,Pais,Oro
16,1900,Paris,France,27
102,1920,Antwerp,Italy,13
112,1924,Paris,France,13
174,1932,Los Angeles,Italy,12
203,1936,Berlin,Germany,33
355,1960,Rome,Italy,13
608,1984,Los Angeles,Italy,14
711,1992,Barcelona,Spain,13
718,1992,Barcelona,Germany,33
780,1996,Atlanta,Italy,13


## 3. Indexación booleana

Replica las consultas 3 y 5 del apartado 2, pero usando indexación booleana en lugar de usar query.

In [10]:
# Consulta 3 
medallero_atenas = (
    data_medallas[
        (data_medallas['Anio'] == 2004) & 
        (data_medallas['Host Ciudad'] == 'Athens')
    ][['Pais','Oro']]
    .sort_values('Oro', ascending=False)
)

medallero_atenas

,Pais,Oro
939,United States,36
942,China,32
941,Russia,28
936,Australia,17
935,Japan,16
...,...,...
960,Mexico,0
964,Colombia,0
965,India,0
966,Paraguay,0


In [11]:
# Consulta 5
#Mostrar las filas de los paises 'Spain', 'France', 'Germany', 'Italy',  que han ganado más de 10 medallas de oro
medallero_df = (
    data_medallas[
        (data_medallas['Pais'].isin(["Spain", "France", "Germany", "Italy"])) & 
        (data_medallas['Oro'] >= 10)  # .isin() es más idiomático que 'in'
    ]
    .sort_values('Oro', ascending=False)
)
medallero_df

,Anio,Host Pais,Host Ciudad,Pais,Oro,Plata,Bronce
718,1992,Spain,Barcelona,Germany,33,21,28
203,1936,Germany,Berlin,Germany,33,26,30
16,1900,France,Paris,France,27,38,37
787,1996,United States,Atlanta,Germany,20,18,27
1188,2016,Brazil,Rio de Janeiro,Germany,17,10,15
1022,2008,China,Beijing,Germany,16,11,14
783,1996,United States,Atlanta,France,15,7,15
608,1984,United States,Los Angeles,Italy,14,6,12
355,1960,Italy,Rome,Italy,13,10,13
860,2000,Australia,Sydney,Germany,13,17,26


## 4. Discretización y variable dummy 


In [12]:
# Copia el dataframe medallas en medallas2 para trabajar con el
medallas2 = data_medallas.copy()
medallas2.head()

,Anio,Host Pais,Host Ciudad,Pais,Oro,Plata,Bronce
0,1896,Greece,Athens,Great Britain,2,3,2
1,1896,Greece,Athens,Hungary,2,1,3
2,1896,Greece,Athens,France,5,4,2
3,1896,Greece,Athens,United States,11,7,2
4,1896,Greece,Athens,Germany,6,5,2


In [13]:
# Genera una nueva columna que sea "puntuacion" asignando:
    # 3 puntos a cada medalla de oro
    # 2 puntos a cada medalla de plata
    # 1 punto a cada medalla de bronce
# Método directo (más eficiente)
medallas2['Puntuacion'] = (
    data_medallas['Oro'] * 3 + 
    data_medallas['Plata'] * 2 + 
    data_medallas['Bronce'] * 1
)

# muestra las primeras filas para comprobar que la nueva columna ha sido creada correctamente
medallas2.head()

,Anio,Host Pais,Host Ciudad,Pais,Oro,Plata,Bronce,Puntuacion
0,1896,Greece,Athens,Great Britain,2,3,2,14
1,1896,Greece,Athens,Hungary,2,1,3,11
2,1896,Greece,Athens,France,5,4,2,25
3,1896,Greece,Athens,United States,11,7,2,49
4,1896,Greece,Athens,Germany,6,5,2,30


In [14]:
# Discretización: Crea una nueva columna "puntuacion_categoria" que divida la puntuación en 3 categorías: baja, media y alta
# Puedes utilizar qcut para generar intervalos de forma equitativa
medallas2["puntuacion_categoria"] = pd.qcut(
    medallas2["Puntuacion"],          # columna numérica
    q=3,                       # 3 cuantiles: terciles
    labels=["baja", "media", "alta"]
)

# muestra las primeras filas para comprobar que la nueva columna ha sido creada correctamente
medallas2.head()

,Anio,Host Pais,Host Ciudad,Pais,Oro,Plata,Bronce,Puntuacion,puntuacion_categoria
0,1896,Greece,Athens,Great Britain,2,3,2,14,media
1,1896,Greece,Athens,Hungary,2,1,3,11,media
2,1896,Greece,Athens,France,5,4,2,25,alta
3,1896,Greece,Athens,United States,11,7,2,49,alta
4,1896,Greece,Athens,Germany,6,5,2,30,alta


In [15]:
# Crea un dataframe "dummydf" con variables dummies para la columna puntuacion_categoria con el prefijo "puntos"
dummydf = pd.get_dummies(medallas2['puntuacion_categoria'],prefix= 'puntos')

# añade las variables dummy al dataframe medallas2 con join 
medallas2 = medallas2.join(dummydf)

# Muestra las primeras filas para comprobar que las variables dummies han sido creadas correctamente
medallas2.head()

,Anio,Host Pais,Host Ciudad,Pais,Oro,Plata,Bronce,Puntuacion,puntuacion_categoria,puntos_baja,puntos_media,puntos_alta
0,1896,Greece,Athens,Great Britain,2,3,2,14,media,False,True,False
1,1896,Greece,Athens,Hungary,2,1,3,11,media,False,True,False
2,1896,Greece,Athens,France,5,4,2,25,alta,False,False,True
3,1896,Greece,Athens,United States,11,7,2,49,alta,False,False,True
4,1896,Greece,Athens,Germany,6,5,2,30,alta,False,False,True


## 5. Union de datasets y consultas con agrupamiento

+ Cada consulta es 0.6 puntos. 
+ La unión de ambos datasets es 1.2 puntos.

In [16]:
# Consulta 1: Usando agrupamiento, mostrar el número total de medallas de oro, plata y bronce de cada país
medallas2.groupby('Pais')[['Oro','Plata','Bronce']].agg('sum').head(10)

,Oro,Plata,Bronce
Pais,,,
Afghanistan,0,0,2
Algeria,5,4,8
Argentina,21,26,30
Armenia,2,8,8
Australasia,3,4,5
Australia,164,173,210
Austria,20,35,41
Azerbaijan,7,14,28
Bahamas,8,2,6


In [17]:
# Importa el dataset paises.csv en el dataframe paises
# Este csv contiene información sobre la población y el PIB per capita de paises, 
    # además del continente y subregion al que pertenecen. 
df_paises = pd.read_csv('paises.csv')

# muestra las primeras filas para comprender su estructura y comprobar la importación
df_paises.head()

,pais,continente,region,poblacion,PIB
0,Afghanistan,Asia,Southern Asia,31056997,700.0
1,Albania,Europe,Southern Europe,3581655,4500.0
2,Algeria,Africa,Northern Africa,32930091,6000.0
3,American Samoa,Oceania,Polynesia,57794,8000.0
4,Andorra,Europe,Southern Europe,71201,19000.0


In [18]:
# Compruba que no haya nulos es el dataframe paises. 
print( df_paises.isnull().sum())

pais          0
continente    0
region        0
poblacion     0
PIB           0
dtype: int64


In [19]:
# Consulta 2: Con agrupamiento, a partir de dataset paises, consultar la media del PIB y poblacición de cada región
df_paises.groupby('region')['PIB'].mean()

region
Australia and New Zealand          25300.000000
Central Asia                        3280.000000
Eastern Asia                       17440.000000
Eastern Europe                      9080.000000
Latin America and the Caribbean     7851.351351
Melanesia                           5520.000000
Micronesia                          7480.000000
Northern Africa                     4866.666667
Northern America                   30900.000000
Northern Europe                    23785.714286
Polynesia                           6157.142857
South-eastern Asia                  6750.000000
Southern Asia                       2766.666667
Southern Europe                    16807.692308
Sub-Saharan Africa                  2420.512821
Western Asia                       10311.764706
Western Europe                     31411.111111
Name: PIB, dtype: float64

In [20]:
# Une los dataframes medallas y paises en un nuevo dataframe llamado medallas_paises usando la columna pais
# Debes hacer un merge interno.
df_paises.columns=['Pais', 'Continente', 'Region', 'Poblacion', 'PIB']
medallas_paises = pd.merge(
    data_medallas,
    df_paises,
    on="Pais",        
    how="inner"       
)


# muestra las primeras filas para comprobar que la unión ha sido correcta
medallas_paises.head()

,Anio,Host Pais,Host Ciudad,Pais,Oro,Plata,Bronce,Continente,Region,Poblacion,PIB
0,1896,Greece,Athens,Hungary,2,1,3,Europe,Eastern Europe,9981334,13900.0
1,1896,Greece,Athens,France,5,4,2,Europe,Western Europe,60876136,27600.0
2,1896,Greece,Athens,United States,11,7,2,Americas,Northern America,298444215,37800.0
3,1896,Greece,Athens,Germany,6,5,2,Europe,Western Europe,82422299,27600.0
4,1896,Greece,Athens,Austria,2,1,2,Europe,Western Europe,8192880,30000.0


In [21]:
# Comprueba que no existan nulos en el dataframe medallas_paises. Si los hay, quizás tienes el ejercicio mal.
print( medallas_paises.isnull().sum())

Anio           0
Host Pais      0
Host Ciudad    0
Pais           0
Oro            0
Plata          0
Bronce         0
Continente     0
Region         0
Poblacion      0
PIB            0
dtype: int64


In [22]:
# Consulta 3: Mostrar el número total de medallas de oro, plata y bronce de cada región, 
    # pero solamente de las regiones con más de 300 medallas de oro
    # La salida debe ser ordenada de forma descentende según el número de medallas de oro

consulta = medallas_paises.groupby("Region")[["Oro", "Plata", "Bronce"]].sum()
consulta = consulta[consulta['Oro'] > 300].sort_values('Oro', ascending=False)
consulta.head()

,Oro,Plata,Bronce
Region,,,
Northern America,1132,941,885
Western Europe,642,736,821
Eastern Europe,623,656,817
Eastern Asia,435,363,372
Northern Europe,390,429,479


## 6. Consultas libres

Realiza 2 consultas adicionales y diferentes a las anteriores, usando query o indexación booleana, que tengan además el uso de  agrupamiento y funciones de agregación y/o creación de intervalos. 

Las consultas se deben hacer sobre el dataset resultante de la unión de ambos csv realizada en el punto 4.

La nota dependerá de la complejidad de la consulta.

In [23]:
# Consulta 1
#Paises que han conseguido una medalla de cada en Barcelona 1992

#query para obtener los datos de Barcelona 1992
barcelona = medallas_paises.query('`Host Ciudad` == "Barcelona"')\
    .dropna(subset=['Oro', 'Plata', 'Bronce'])

consulta1 = barcelona.groupby('Pais')[['Oro', 'Plata', 'Bronce']].sum()
consulta1 = consulta1[(consulta1['Oro'] >= 1) & 
                    (consulta1['Plata'] >= 1) & 
                    (consulta1['Bronce'] >= 1)]
consulta1.head()

,Oro,Plata,Bronce
Pais,,,
Australia,7,9,11
Bulgaria,3,7,6
Canada,7,4,7
China,16,22,16
Cuba,14,6,11


In [24]:
# Consulta 2
#Paises con numero de medallistas olimpicos
consulta2 = medallas_paises.groupby('Pais')[['Oro', 'Plata', 'Bronce']].sum()
consulta2['Medallistas'] = consulta2['Oro'] + consulta2['Plata'] + consulta2['Bronce']
consulta2.head()

,Oro,Plata,Bronce,Medallistas
Pais,,,,
Afghanistan,0,0,2,2
Algeria,5,4,8,17
Argentina,21,26,30,77
Armenia,2,8,8,18
Australia,164,173,210,547


In [25]:
# Source - https://stackoverflow.com/a/16923367
# Posted by Andy Hayden, modified by community. See post 'Timeline' for change history
# Retrieved 2026-02-09, License - CC BY-SA 4.0

medallas_paises.to_csv("medallas_paises.csv", sep='\t', encoding='utf-8', index=False, header=True)
